&copy; 2019 by Pearson Education, Inc. All Rights Reserved. The content in this notebook is based on the book [**Python for Programmers**](https://amzn.to/2VvdnxE).

# 9. Files and Exceptions
* Note: Some sections reordered from our book for _Python Full Throttle_ presentation purposes. 

# 9.1 Introduction
* Text files processing.
* **CSV files**&mdash;common format for machine-learning datasets.
* Serialize objects into the **JSON data-interchange format**—commonly used to transmit over the Internet—and deserialize JSON into objects.
* **`with` statement** for avoiding “resource leaks.”
* **`try`...`except`...`else`...`finally` statement** for exception handling.
* **`raise` exceptions** to indicate runtime problems.

# 9.2 Files 
* Python views a **text file** as a sequence of characters and a **binary file** (for images, videos and more) as a sequence of bytes starting at position 0. 

![Conceptual view of a file](ch09images/AAEMYSR0.png "Conceptual view of a file")

### Standard File Objects Like Other C-Based Languages
* When a Python program begins execution, it creates three **standard file objects**:
    * **`sys.stdin`** — the **standard input file object**
    * **`sys.stdout`** — the **standard output file object**, and 
    * **`sys.stderr`** — the **standard error file object**. 

## 9.3 Text-File Processing
* Python imposes no structure on a file.

## 9.3.1 Writing to a Text File: Introducing the `with` Statement 
* For each file you **open**, Python creates a **file object** that you’ll use to interact with the file.
* `with` statement 
    * Acquires a resource and assigns its corresponding object to a variable.
    * Allows the application to use the resource via that variable.
    * Calls the resource object’s **`close` method** to release the resource.

In [ ]:
with open('accounts.txt', mode='w') as accounts:
    accounts.write('100 Jones 24.98\n')
    accounts.write('200 Doe 345.67\n')
    accounts.write('300 White 0.00\n')
    accounts.write('400 Stone -42.16\n')
    accounts.write('500 Rich 224.62\n')

* Can also **write to a file with `print`**, which **automatically outputs a `\n`**, as in
>```python
print('100 Jones 24.98', file=accounts)
```

### Contents of accounts`.txt` File 

In [ ]:
!cat accounts.txt

## 9.3.2 Reading Data from a Text File
* Iterating through a file object, **reads one line at a time from the file** and returns it as a string. 

In [ ]:
with open('accounts.txt', mode='r') as accounts:
    print(f'{"Account":<10}{"Name":<10}{"Balance":>10}')
    for record in accounts:
        account, name, balance = record.split()
        print(f'{account:<10}{name:<10}{balance:>10}')

# 9.5 Serialization with JSON and the Python Standard Library Module `json` 
* **JSON (JavaScript Object Notation)** is a text-based, human-and-computer-readable, data-interchange format used to represent objects as collections of name–value pairs. 
* Preferred data format for transmitting objects across platforms. 

### JSON Data Format Is Similar to Python Dictionaries 
* Each JSON object contains a comma-separated list of **property names** and **values**, in curly braces. 
> ```python
{"account": 100, "name": "Jones", "balance": 24.98}
```

* JSON arrays, like Python lists, are comma-separated values in square brackets. 
> ```python
[100, 200, 300]
```

* Values in JSON objects and arrays can be:
    * **strings** in **double quotes**
    * **numbers**
    * JSON Boolean values **`true`** or **`false`** 
    * **`null`** (like `None` in Python)
    * **arrays**  
    * **other JSON objects**

In [ ]:
accounts_dict = {
    'accounts': [
        {'account': 100, 'name': 'Jones', 'balance': 24.98},
        {'account': 200, 'name': 'Doe', 'balance': 345.67}]
}

### Serializing an Object to JSON with the `json` Module’s **`dump` Function** 

In [ ]:
import json

In [ ]:
with open('accounts.json', 'w') as accounts:
    json.dump(accounts_dict, accounts)

In [ ]:
!cat accounts.json

### Deserializing JSON Text with the `json` Module’s **`load` Function** 
* Reads entire file and converts the JSON into a Python object. 

In [ ]:
with open('accounts.json', 'r') as accounts:
    accounts_json = json.load(accounts)

### Interacting with the Deserialized Object

In [ ]:
type(accounts_json)

In [ ]:
accounts_json

In [ ]:
accounts_json['accounts']

In [ ]:
accounts_json['accounts'][0]['name']

In [ ]:
accounts_json['accounts'][1]

### Pretty Printing JSON Text with the `json` Module’s **`dumps` Function**
* Short for “dump string”. 
* Returns a Python string representation of an object in JSON format. 

In [ ]:
with open('accounts.json', 'r') as accounts:
    print(json.dumps(json.load(accounts), indent=4))

### Serializing/Deserializing Objects of Custom Class Types
* Check out the [open source jsons library](https://jsons.readthedocs.io/en/latest/?badge=latest).

# 9.12 Working with CSV Files 
* Datasets are often provided in **CSV (comma-separated values)** format. 

## 9.12.1 Python Standard Library Module `csv` 
* Provides functions for working with CSV files. 

### Writing to a CSV File with the  **`writer` Function**
* **Open CSV files with the additional keyword argument `newline=''`** to ensure that **newlines are processed properly** across platforms.
* Each call to a `writer`’s **`writerow` method** receives an **iterable** to write as a line of comma-delimited text in the file.
* `writerow` delimits values with commas, but you can specify custom delimiters. 

In [ ]:
import csv

In [ ]:
with open('accounts.csv', mode='w', newline='') as accounts:
    writer = csv.writer(accounts)  # object that writes CSV records
    writer.writerow([100, 'Jones', 24.98])
    writer.writerow([200, 'Doe', 345.67])
    writer.writerow([300, 'White', 0.00])
    writer.writerow([400, 'Stone', -42.16])
    writer.writerow([500, 'Rich', 224.62])

In [ ]:
!cat accounts.csv

* Multiple `writerow` calls can be replaced with one **`writerows`** call that outputs a **comma-separated list of iterables** (e.g., a list of lists) representing the records.

### Fields Containing Commas
* If you write data that contains commas within a given string, `writerow` encloses that string in double quotes.
* The field `'Jones, Sue'` would be written as `"Jones, Sue"` and read as a single field on input.

### Reading from a CSV File with the **`reader`** Function
* Returns an object that **reads CSV-format data** from the specified file object. 
* Can iterate through a `reader` object to read one record at a time. 

In [ ]:
with open('accounts.csv', 'r', newline='') as accounts:
    print(f'{"Account":<10}{"Name":<10}{"Balance":>10}')
    reader = csv.reader(accounts)
    for record in reader:  
        account, name, balance = record
        print(f'{account:<10}{name:<10}{balance:>10}')

# 9.7 Additional Notes Regarding Files
### File-Open Modes


| Mode | Description
| :------ | :------
| `'r'` | Open a text file for reading. This is the default if you do not specify the file-open mode when you call open. 
| `'w'` | Open a text file for writing. Existing file contents are _deleted_. 
| **`'a'`** | Open a text file for appending at the end, creating the file if it does not exist. New data is written at the end of the file. 
| **`'r+'`** | Open a text file reading and writing. 
| **`'w+'`** | Open a text file reading and writing. Existing file contents are _deleted_.
| **`'a+'`** | Open a text file reading and appending at the end. New data is written at the end of the file. If the file does not exist, it is created. 

### Other File Object Methods
* The classes that Python uses to create file objects are defined in the Python Standard Library’s [**io module**](https://docs.python.org/3/library/io.html). See the documentation for many additional file-object methods.

# 9.8 Handling Exceptions
Exceptions when you work with files: 
* A **`FileNotFoundError`** occurs if you **attempt to open a non-existent file** for reading with the `'r'` or `'r+'` modes. 
* A **`PermissionsError`** occurs if you **attempt an operation for which you do not have permission**. 
* A `ValueError` (with the error message `'I/O operation on closed file.'`) occurs when you **attempt to write to a file that has already been closed**.

## 9.8.2 `try` Statements

```python 
# dividebyzero.py
"""Simple exception handling example."""

while True:
    # attempt to convert and divide values
    try:
        number1 = int(input('Enter numerator: '))
        number2 = int(input('Enter denominator: '))
        result = number1 / number2
    except ValueError:  # tried to convert non-numeric value to int
        print('You must enter two integers\n')
    except ZeroDivisionError:  # denominator was 0
        print('Attempted to divide by zero\n')
    else:  # executes only if no exceptions occur
        print(f'{number1:.3f} / {number2:.3f} = {result:.3f}')
        break  # terminate the loop
```

In [ ]:
run dividebyzero.py

## 9.8.3 Catching Multiple Exceptions in One `except` Clause
```python
except (type1, type2, …) as variable_name:
```

* **`as` clause is optional**&mdash;needed only if you want to access the exception object in the `except` suite.

# 9.9 `finally` Clause of the `try` Statement
* Placed after any `except` clauses or the `else` clause. 
* Guaranteed to execute. 
* In other languages, this makes the `finally` suite an ideal location to place resource-deallocation code. 
* In Python, the `with` statement is preferred for this.

### Demonstrating that the `finally` Clause Always Executes

In [ ]:
try:
    print('try suite with no exceptions raised')
except:
    print('this will not execute')
else:
    print('else executes because no exceptions in the try suite')
finally:  
    print('finally always executes')

In [ ]:
try:
    print('try suite that raises an exception')
    int('hello')
    print('this will not execute')
except ValueError:
    print('a ValueError occurred')
else:
    print('else will not execute because an exception occurred')
finally:  
    print('finally always executes')

### Combining `with` Statements and `try…except` Statements 
* Most resources that require **explicit release**, such as files, network connections and database connections, have potential exceptions associated with processing those resources. 
* Can place a `with` statement in a `try` suite to guarantee that the resource gets released properly and have the opportunity to catch exceptions. 

# 9.10 Explicitly Raising an Exception with the **`raise`** Statement 
```python
raise ExceptionClassName
```

* Creates an object of the specified exception class. 
* Optionally, specify parentheses containing arguments to initialize the exception object.
* Code that raises an exception should first release any resources acquired before the exception occurred. 
* Generally, you should raise one of Python’s [**many built-in exception types**](https://docs.python.org/3/library/exceptions.html).

# 9.11 Stack Unwinding and Tracebacks
* Each exception object stores information indicating the precise series of function calls that led to the exception. 
* In a traceback, the bottom of the stack is shown first, and the **raise point** is shown last.

In [ ]:
def function1():
    function2()

In [ ]:
def function2():
    raise Exception('An exception occurred')

In [ ]:
function1()

### Tip for Reading Tracebacks
* When reading a traceback, start from the end of the traceback and read the error message first. 
* Then, read upward through the traceback, looking for the first line that indicates code you wrote in your program. 
* Typically, this is the location in your code that led to the exception.

# More Info 
* See Lesson 9 in [**Python Fundamentals LiveLessons** here on O'Reilly Online Learning](https://learning.oreilly.com/videos/python-fundamentals/9780135917411)
* See Chapter 9 in [**Python for Programmers** on O'Reilly Online Learning](https://learning.oreilly.com/library/view/python-for-programmers/9780135231364/)
* Interested in a print book? Check out:

| Python for Programmers | Intro to Python for Computer<br>Science and Data Science
| :------ | :------
| <a href="https://amzn.to/2VvdnxE"><img alt="Python for Programmers cover" src="../images/PyFPCover.png" width="150" border="1"/></a> | <a href="https://amzn.to/2LiDCmt"><img alt="Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud" src="../images/IntroToPythonCover.png" width="159" border="1"></a>

>Please **do not** purchase both books&mdash;_Python for Programmers_ is a subset of _Intro to Python for Computer Science and Data Science_

&copy; 2019 by Pearson Education, Inc. All Rights Reserved. The content in this notebook is based on the book [**Python for Programmers**](https://amzn.to/2VvdnxE).